In [2]:
import sys

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
class TreeNode:
    def __init__(self, val, left = None, right = None, parent = None):
        self.val = val
        self.left = left
        self.right = right
        self.parent = parent
        
    @classmethod
    def construct_from_array(cls, arr, i, size):
        root = TreeNode(arr[i])
        if i <= size//2:
            if 2 * i <=  size and arr[2 * i]:
                root.left = cls.construct_from_array(arr, 2*i, size)
            if 2 * i + 1 <= size and arr[2 * i + 1]:
                root.right = cls.construct_from_array(arr, 2*i +1, size)
        return root
    
    def display(self):

        # No child.
        if not self.left and not self.right:
            line = str(self.val)
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        elif self.left and not self.right:
            lines, n, p, x = self.left.display()
            s = str(self.val)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        elif not self.left and self.right:
            lines, n, p, x = self.right.display()
            s = str(self.val)
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        else:
            left, n, p, x = self.left.display()
            right, m, q, y = self.right.display()
            s = str(self.val)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
            second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
            if p < q:
                left += [n * ' '] * (q - p)
            elif q < p:
                right += [m * ' '] * (p - q)
            zipped_lines = zip(left, right)
            lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
            return lines, n + m + u, max(p, q) + 2, n + u // 2

In [41]:
class MinHeap:

    def __init__(self, maxsize, print_step= True):

        self.maxsize = maxsize
        self.size = 0
        self.Heap = [None] * (self.maxsize + 1)
        self.Heap[0] = -1e15
        self.FRONT = 1
        self.print_step = print_step

    # Function to return the position of
    # parent for the node currently
    # at pos
    def parent(self, pos):

        return pos // 2

    # Function to return the position of
    # the left child for the node currently
    # at pos
    def leftChild(self, pos):

        return 2 * pos

    # Function to return the position of
    # the right child for the node currently
    # at pos
    def rightChild(self, pos):

        return (2 * pos) + 1

    # Function that returns true if the passed
    # node is a leaf node
    def isLeaf(self, pos):

        if pos > (self.size//2) and pos <= self.size:
            return True
        return False

    # Function to swap two nodes of the heap
    def swap(self, fpos, spos):

        self.Heap[fpos], self.Heap[spos] = (self.Heap[spos],
                                            self.Heap[fpos])

    # Function to heapify the node at pos
    def minHeapify(self, pos):

        # If the node is a non-leaf node and smaller
        # than any of its child
        if self.Heap[pos] and not self.isLeaf(pos):
            if (self.Heap[pos] > self.Heap[self.leftChild(pos)] or
                self.Heap[self.rightChild(pos)] and self.Heap[pos] > self.Heap[self.rightChild(pos)]):

                # Swap with the left child and heapify
                # the left child
                if (self.Heap[self.rightChild(pos)] and self.Heap[self.leftChild(pos)] >
                    self.Heap[self.rightChild(pos)]):
                    if self.print_step:
                        print(f'\n\nSwapping {self.Heap[pos]} and {self.Heap[self.rightChild(pos)]}\n')
                    self.swap(pos, self.rightChild(pos))
                    if self.print_step:
                        self.display()
                    self.minHeapify(self.rightChild(pos))

                # Swap with the right child and heapify
                # the right child
                else:
                    if self.print_step:
                        print(f'\n\nSwapping {self.Heap[pos]} and {self.Heap[self.leftChild(pos)]}\n')
                    self.swap(pos, self.leftChild(pos))
                    if self.print_step:
                        self.display()
                    self.minHeapify(self.leftChild(pos))
                    
                

    # Function to insert a node into the heap
    def insert(self, element):
        if self.print_step:
            print(f'\n\nInserting {element}\n')

        if self.size >= self.maxsize:
            return
        self.size += 1
        self.Heap[self.size] = element

        current = self.size
        if self.print_step:
            self.display()
            
        if self.Heap[current] < self.Heap[self.parent(current)]:
            print('\nPerforming Up-heap bubbling') 
            
        while (self.Heap[current] <
            self.Heap[self.parent(current)]):
            if self.print_step:
                print(f'\n\nSwapping {self.Heap[current]} and {self.Heap[self.parent(current)]}\n')
            self.swap(current, self.parent(current))
            current = self.parent(current)
            if self.print_step:
                self.display()
        

    # Function to print the contents of the heap
    def Print(self):

        for i in range(1, (self.size // 2) + 1):
            print(" PARENT : " + str(self.Heap[i]) +
                " LEFT CHILD : " + str(self.Heap[2 * i]) +
                " RIGHT CHILD : " + str(self.Heap[2 * i + 1]))

    # Function to remove and return the minumum
    # element from the heap
    def extractMin(self):        
        popped = self.Heap[self.FRONT]
        if self.print_step:
            print('Heap is\n')
            self.display()
            print(f'\nExtracting Min element {popped}\n')
        self.Heap[self.FRONT] = self.Heap[self.size]
        self.Heap[self.size] = None
        self.size -= 1
        if self.print_step:
            print(f'\nPlacing {self.Heap[self.FRONT]} at root\n')
            self.display()
        self.minHeapify(self.FRONT)

        return popped
    
    def display(self):
        root = TreeNode.construct_from_array(self.Heap, self.FRONT, self.size)        
        lines, *_ = root.display()
        for line in lines:
            print(line)
            
    def heap_sort(self):
        self._sorted_array = self.Heap[1:self.size + 1]
        print('Input Min Heap Array:\n')
        print(self._sorted_array)
        print()
        self.display()
        print('\nApplying heap-sort on min heap to sort in descending order')
        for i in range(self.size, 0, -1):
            min_el = self.extractMin()
            self._sorted_array[i-1] = min_el
            temp = self.Heap[1:self.size + 1]
            self._sorted_array =  temp + self._sorted_array[i-1:]
            print(f'\nSorted array after removal of {min_el}: {self._sorted_array}\n')
            print('-----------------------------------------\n')
        print(f'\n\nFinal Sorted array:\n{self._sorted_array}')
        return self._sorted_array
    
        
    @classmethod
    def construct_from_list(cls, elements, print_step=True):
        minHeap = MinHeap(2*len(elements), print_step)
        for e in elements:
            minHeap.insert(e) 
            print('-----------------------------------------')
        return minHeap
    
    @classmethod
    def construct_from_tree(cls, elements, print_step=True):
        bt_elements = [-1e15]
        bt_elements.extend(elements)
        #bt_elements.extend([None, None])
        #print(bt_elements)
        root = TreeNode.construct_from_array(bt_elements, 1, len(elements))        
        lines, *_ = root.display()
        print('Input Binary tree\n')
        for line in lines:
            print(line)
        minHeap = MinHeap(2*len(elements), print_step)
        minHeap.Heap = bt_elements
        minHeap.size = len(bt_elements) -1
        #print(maxHeap.Heap)
        print('\nPerforming min-heapification on internal nodes\n')
        for i in range( minHeap.size//2, 0, -1):           
            cur_node = minHeap.Heap[i]
            print(f'\n\nMin Heapify Node: {cur_node}')
            minHeap.minHeapify(i) 
            print('-----------------------------------------')

        return minHeap
    


In [42]:
min_heap = MinHeap.construct_from_list([10,22,34,56,13,84,7,9,8,1], print_step=True)



Inserting 10

10
-----------------------------------------


Inserting 22

  10
 /  
22  
-----------------------------------------


Inserting 34

  10_ 
 /   \
22  34
-----------------------------------------


Inserting 56

    10_ 
   /   \
  22  34
 /      
56      
-----------------------------------------


Inserting 13

    __10_ 
   /     \
  22_   34
 /   \    
56  13    

Performing Up-heap bubbling


Swapping 13 and 22

    __10_ 
   /     \
  13_   34
 /   \    
56  22    
-----------------------------------------


Inserting 84

    __10___ 
   /       \
  13_     34
 /   \   /  
56  22  84  
-----------------------------------------


Inserting 7

    __10___  
   /       \ 
  13_     34 
 /   \   /  \
56  22  84  7

Performing Up-heap bubbling


Swapping 7 and 34

    __10__   
   /      \  
  13_     7_ 
 /   \   /  \
56  22  84 34


Swapping 7 and 10

    __7___   
   /      \  
  13_    10_ 
 /   \  /   \
56  22 84  34
-----------------------------------------


In

In [16]:
min_heap.extractMin()

Heap is

      __7___   
     /      \  
   __8_    10_ 
  /    \  /   \
  9_  22 84  34
 /  \          
56 13          

Extracting Min element 7


Placing 13 at root

    __13___   
   /       \  
   8_     10_ 
  /  \   /   \
  9 22  84  34
 /            
56            


Swapping 13 and 8

     __8___   
    /      \  
   13_    10_ 
  /   \  /   \
  9  22 84  34
 /            
56            


Swapping 13 and 9

     __8___   
    /      \  
    9_    10_ 
   /  \  /   \
  13 22 84  34
 /            
56            


7

In [18]:
min_heap.insert(10)



Inserting 10

       __8___   
      /      \  
    __9_    10_ 
   /    \  /   \
  13_  22 84  34
 /   \          
56  10          


Swapping 10 and 13

       __8___   
      /      \  
    __9_    10_ 
   /    \  /   \
  10_  22 84  34
 /   \          
56  13          


In [22]:
min_heap.insert(57)



Inserting 57

       ______8_____   
      /            \  
    __9___        10_ 
   /      \      /   \
  10_    22_    84  34
 /   \  /   \  /      
56  13 57  57 57      


Swapping 57 and 84

       ______8_____   
      /            \  
    __9___        10_ 
   /      \      /   \
  10_    22_    57  34
 /   \  /   \  /      
56  13 57  57 84      


In [25]:
min_heap = MinHeap.construct_from_tree([1,22,34,56,13,84,7,9,3, 10,15], print_step=True)

Input Binary tree

      ______1___  
     /          \ 
   _22___      34 
  /      \    /  \
 56     13_  84  7
/  \   /   \      
9  3  10  15      

Performing min-heapification on internal nodes



Min Heapify Node: 13


Swapping 13 and 10

      ______1___  
     /          \ 
   _22___      34 
  /      \    /  \
 56     10_  84  7
/  \   /   \      
9  3  13  15      
-----------------------------------------


Min Heapify Node: 56


Swapping 56 and 3

      ______1___  
     /          \ 
  __22___      34 
 /       \    /  \
 3_     10_  84  7
/  \   /   \      
9 56  13  15      
-----------------------------------------


Min Heapify Node: 34


Swapping 34 and 7

      ______1__   
     /         \  
  __22___      7_ 
 /       \    /  \
 3_     10_  84 34
/  \   /   \      
9 56  13  15      
-----------------------------------------


Min Heapify Node: 22


Swapping 22 and 3

      ______1__   
     /         \  
   __3___      7_ 
  /      \    /  \
 22_    10_  84 34
/ 

In [43]:
min_heap.heap_sort()

Input Min Heap Array:

[1, 7, 10, 9, 8, 84, 34, 56, 13, 22]

      ___1___   
     /       \  
   __7__    10_ 
  /     \  /   \
  9_    8 84  34
 /  \  /        
56 13 22        

Applying heap-sort on min heap to sort in descending order
Heap is

      ___1___   
     /       \  
   __7__    10_ 
  /     \  /   \
  9_    8 84  34
 /  \  /        
56 13 22        

Extracting Min element 1


Placing 22 at root

      _22___   
     /      \  
   __7     10_ 
  /   \   /   \
  9_  8  84  34
 /  \          
56 13          


Swapping 22 and 7

       _7___   
      /     \  
   __22    10_ 
  /    \  /   \
  9_   8 84  34
 /  \          
56 13          


Swapping 22 and 8

      __7___   
     /      \  
   __8_    10_ 
  /    \  /   \
  9_  22 84  34
 /  \          
56 13          

Sorted array after removal of 1: [7, 8, 10, 9, 22, 84, 34, 56, 13, 1]

-----------------------------------------

Heap is

      __7___   
     /      \  
   __8_    10_ 
  /    \  /   \
  9_  22 84  34
 /

[84, 56, 34, 22, 13, 10, 9, 8, 7, 1]